In [ ]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta

In [50]:
# Importing and defining a variable for each spreadsheet 

folder_name = '2021_08_ago'

df1 = pd.read_excel(folder_name + "\Alstom (1).xlsx")
df2 = pd.read_excel(folder_name + "\Alstom (2).xlsx")
df3 = pd.read_excel(folder_name + "\Alstom (3).xlsx")
df4 = pd.read_excel(folder_name + "\Alstom (4).xlsx")
df5 = pd.read_excel(folder_name + "\Alstom (5).xlsx")
df6 = pd.read_excel(folder_name + "\Alstom (6).xlsx")
df7 = pd.read_excel(folder_name + "\Alstom (7).xlsx")
df8 = pd.read_excel(folder_name + "\Alstom (8).xlsx")
df9 = pd.read_excel(folder_name + "\Alstom (9).xlsx")
df10 = pd.read_excel(folder_name + "\Alstom (10).xlsx")
df11 = pd.read_excel(folder_name + "\Alstom (11).xlsx")
df12 = pd.read_excel(folder_name + "\Alstom (12).xlsx")
df13 = pd.read_excel(folder_name + "\Alstom (13).xlsx")
df14 = pd.read_excel(folder_name + "\Alstom (14).xlsx")
# df15 = pd.read_excel(folder_name + "\Alstom (15).xlsx")

In [3]:
# Appending all spreadsheets into one and counting the number of rows  

df = df1.append(df2, ignore_index = True)
df = df.append(df3, ignore_index = True)
df = df.append(df4, ignore_index = True)
df = df.append(df5, ignore_index = True)
df = df.append(df6, ignore_index = True)
df = df.append(df7, ignore_index = True)
df = df.append(df8, ignore_index = True)
df = df.append(df9, ignore_index = True)
df = df.append(df10, ignore_index = True)
df = df.append(df11, ignore_index = True)
df = df.append(df12, ignore_index = True)
df = df.append(df13, ignore_index = True)
df = df.append(df14, ignore_index = True)
# df = df.append(df15, ignore_index = True)
print('%s is the total number of lines in this spreadsheet' %len(df))

10232 is the total number of lines in this spreadsheet


In [4]:
##Changing columns names.

df.rename(columns = {"respDescription" : "Responsability",
                     "Power Plant" : "Power_plant", 
                     "Element" : "WTG",
                     "Downtime" : "Downtime_type",
                     "Time from" : "Time_from",
                     "Time to" : "Time_to"
                    }, inplace = True)

In [6]:
# Adjusting the values in each column for the right data type.

df["Responsability"] = df["Responsability"].astype("category")
df["Power_plant"] = df["Power_plant"].astype("category")
df["WTG"] = df["WTG"].astype("category")
df["Downtime_type"] = df["Downtime_type"].astype("category")
df["Time_from"] = pd.to_datetime(df["Time_from"])
df["Time_to"] = pd.to_datetime(df["Time_to"])

In [14]:
## changing time before the working month to the first day/hour/second of 
## the month to study only the analysed month.

time = dt.datetime.now()
time_to = time.replace( day=1, hour=0, minute=0, second=0, microsecond=0)
time_from = time_to - relativedelta(months=1)
# print('time from: %s' %time_from)
# print('time to  : %s' %time_to)


mask = df["Time_from"]  <= time_from
df.loc[mask, "Time_from"] = time_from
# df["Time_from"].min()


## changing time before the working month to the first day/hour/second of 
## the month to study only the analysed month.

mask2 = df["Time_to"]  >= time_to
df.loc[mask2, "Time_to"] = time_to
# df["Time_to"].max()

Timestamp('2021-09-01 00:00:00')

In [16]:
## Converting timedelta "Duration" column to a float number.

df["Duration"] = df["Time_to"] - df["Time_from"]
mask3 = df["Duration"]
df["Duration"] = mask3.dt.total_seconds()/3600

In [17]:
## separating the alarm code column into categories: alarm number, code and description.

df[["Alarm_number", "Alarm_code","Alarm_descrition"]] = df.Alarm.str.split(" - ", expand = True)
df.drop(columns = "Alarm", inplace = True)

In [18]:
## filling na with corrective maintenance code and setting it as integer.

df["Alarm_number"] = df["Alarm_number"].fillna("40")
df["Alarm_number"] = df["Alarm_number"].astype("int")

In [23]:
## opening up the Alstom Codes spreadsheet as reference for system and alarm correlation.

alstom_codes = pd.read_excel("alstom_codes.xlsx")
# alstom_codes.info()

In [33]:
## Joining system by alarms in the df.

df = df.merge(alstom_codes, how = "inner",  left_on = "Alarm_number", right_on = "number" )
df.drop(columns = "number", inplace = True)

In [35]:
## Joining System stops by alarms to the table

alstom_powerplant = pd.read_excel("alstom_powerplant.xlsx")
# alstom_powerplant.info()

df = df.merge(alstom_powerplant, how = "left",  left_on = "Power_plant", right_on = "Power_plants")
df.drop(columns = "Power_plants", inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Complex       15 non-null     object
 1   Power_plants  15 non-null     object
dtypes: object(2)
memory usage: 368.0+ bytes


In [37]:
## adjusting the data types of some columns.

df["Power_plant"] = df["Power_plant"].astype("category")
df["system"] = df["system"].astype("category")
df["Complex"] = df["Complex"].astype("category")
df["Status"] = df["Status"].astype("category")

In [39]:
## organizing the df to the new index.

df.sort_values(["Power_plant","WTG","Time_from"], ascending=[True, True, True], inplace=True)
df.reset_index(drop = True, inplace = True)

In [46]:
## creating the output file with the new dataframe.

file_name = 'complete_list_alston_' + time_from.strftime("%m") + '.xlsx'
excel_file = pd.ExcelWriter(file_name)
df.to_excel(excel_file, sheet_name = "Alstom", index=False, freeze_panes = (1,0))
excel_file.save()